In [20]:
E_L = 400 #GPa
E_L = 400e9 #Pa
E_T = 50 #GPa
E_T = 50e9 #Pa
nu_LT = 0.1
# nu_TL = E_T/E_L*nu_LT
nu_TL = E_T/E_L*nu_LT
G_LT = 40 #GPa
G_LT = 40e9 #Pa
t = 0.1 #mm
t = 0.1e-3 #m
angle_list = [0, 45, -45, 90]
repetitions = 1

nu_TL

0.0125

In [21]:
#A laminate is consisting of a ply stack of [0/45/90/-45]2. Material properties of a ply is given right. The ply thickness is 0.1 mm
#a) Calculate the A matrix, D matrix, and B matrix of this laminate.
import numpy as np
import sympy as sym
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)



# Q matrix
Q = [[E_L/(1-nu_LT*nu_TL), E_L*nu_TL/(1-nu_LT*nu_TL), 0],
     [E_L*nu_TL/(1-nu_LT*nu_TL), E_T/(1-nu_LT*nu_TL), 0],
     [0, 0, G_LT]]
display(Q)
T_sigma = lambda theta: [[np.cos(theta)**2, np.sin(theta)**2, 2*np.cos(theta)*np.sin(theta)],
                   [np.sin(theta)**2, np.cos(theta)**2, -2*np.cos(theta)*np.sin(theta)],
                   [-np.cos(theta)*np.sin(theta), np.cos(theta)*np.sin(theta), np.cos(theta)**2-np.sin(theta)**2]]
T_epsilon = lambda theta: [[np.cos(theta)**2, np.sin(theta)**2, np.cos(theta)*np.sin(theta)],
                     [np.sin(theta)**2, np.cos(theta)**2, -np.cos(theta)*np.sin(theta)],
                     [-2*np.cos(theta)*np.sin(theta), 2*np.cos(theta)*np.sin(theta), np.cos(theta)**2-np.sin(theta)**2]]
def Q_bar(theta):
    T_sigma_ = T_sigma(theta)
    T_epsilon_ = T_epsilon(theta)
    return np.dot(np.dot(np.linalg.inv(T_sigma_), Q), T_epsilon_)

# Convert to radians
angle_list = [angle*np.pi/180 for angle in angle_list]
Q_list = []
for angle in angle_list:
    print(f"Q_bar({angle})")
    display(sym.Matrix(Q_bar(angle)))
    Q_list.append(Q_bar(angle))
    print()

display(sym.Matrix(Q_list))


[[400500625782.2278, 5006257822.277847, 0],
 [5006257822.277847, 50062578222.77847, 0],
 [0, 0, 40000000000.0]]

Q_bar(0.0)


Matrix([
[400500625782.228, 5006257822.27785,             0],
[5006257822.27785, 50062578222.7785,             0],
[               0,                0, 40000000000.0]])


Q_bar(0.7853981633974483)


Matrix([
[155143929912.391, 75143929912.3905, 87609511889.8623],
[75143929912.3905,  155143929912.39, 87609511889.8623],
[87609511889.8623, 87609511889.8623, 110137672090.113]])


Q_bar(-0.7853981633974483)


Matrix([
[ 155143929912.391,  75143929912.3905, -87609511889.8623],
[ 75143929912.3905,   155143929912.39, -87609511889.8623],
[-87609511889.8623, -87609511889.8623,  110137672090.113]])


Q_bar(1.5707963267948966)


Matrix([
[   50062578222.7785,    5006257822.27785, 2.13968326859788e-6],
[   5006257822.27785,    400500625782.228, 1.93184583935585e-5],
[2.13968326859788e-6, 1.93184583935585e-5,       40000000000.0]])

Matrix([
[                [400500625782.228, 5006257822.27785, 0.0],                 [5006257822.27785, 50062578222.7785, 0.0],                                 [0.0, 0.0, 40000000000.0]],
[   [155143929912.391, 75143929912.3905, 87609511889.8623],     [75143929912.3905, 155143929912.39, 87609511889.8623],    [87609511889.8623, 87609511889.8623, 110137672090.113]],
[  [155143929912.391, 75143929912.3905, -87609511889.8623],    [75143929912.3905, 155143929912.39, -87609511889.8623],  [-87609511889.8623, -87609511889.8623, 110137672090.113]],
[[50062578222.7785, 5006257822.27785, 2.13968326859788e-6], [5006257822.27785, 400500625782.228, 1.93184583935585e-5], [2.13968326859788e-6, 1.93184583935585e-5, 40000000000.0]]])

In [22]:
Q_list = []
for angle in angle_list:
    print(f"Q_bar({angle})")
    display(sym.Matrix(Q_bar(angle)))
    Q_list.append(Q_bar(angle))
    print()

def BD_func(symmetric=False, repetitions=1, t=1):
    D = 0
    B = 0
    m=0
    if not symmetric:
        for i in np.arange(repetitions/2 * len(angle_list), -repetitions/2 * len(angle_list), -1):
            B += ((t*i)**2-(t*(i-1))**2)/2 * Q_list[m]
            D += ((t*i)**3-(t*(i-1))**3)/3 * Q_list[m]
            m += 1
            if m == len(angle_list):
                m = 0
        
    if symmetric:
        B = np.zeros((3, 3))
        repetitions = repetitions*2
        for i in np.arange(len(angle_list), 0, -1):
            D += ((t*i)**3-(t*(i-1))**3)/3 * Q_list[m]
            m += 1
            if m == len(angle_list):
                m = 0

        if m == 0:
                m = len(angle_list)-1
                
        for i in np.arange(0, -len(angle_list), -1):
            D += ((t*i)**3-(t*(i-1))**3)/3 * Q_list[m]
            m -= 1
            if m == -len(angle_list):
                m = 0
                
    return B, D

B, D = BD_func(symmetric=True, repetitions=1, t=1)
display(sym.Matrix(B))
display(sym.Matrix(D))

Q_bar(0.0)


Matrix([
[400500625782.228, 5006257822.27785,             0],
[5006257822.27785, 50062578222.7785,             0],
[               0,                0, 40000000000.0]])


Q_bar(0.7853981633974483)


Matrix([
[155143929912.391, 75143929912.3905, 87609511889.8623],
[75143929912.3905,  155143929912.39, 87609511889.8623],
[87609511889.8623, 87609511889.8623, 110137672090.113]])


Q_bar(-0.7853981633974483)


Matrix([
[ 155143929912.391,  75143929912.3905, -87609511889.8623],
[ 75143929912.3905,   155143929912.39, -87609511889.8623],
[-87609511889.8623, -87609511889.8623,  110137672090.113]])


Q_bar(1.5707963267948966)


Matrix([
[   50062578222.7785,    5006257822.27785, 2.13968326859788e-6],
[   5006257822.27785,    400500625782.228, 1.93184583935585e-5],
[2.13968326859788e-6, 1.93184583935585e-5,       40000000000.0]])

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

Matrix([
[12601551939924.9, 1429319983312.47, 700876095118.899],
[1429319983312.47, 4191038798498.12, 700876095118.899],
[700876095118.898, 700876095118.898, 2922386316228.62]])

In [23]:
# A matrix
A = 2*t*(sum(Q_list))
display(sym.Matrix(A))

Matrix([
[    152170212.765957,   32060075.0938673, 4.27936653719576e-10],
[    32060075.0938673,   152170212.765957,   3.8636916787117e-9],
[4.27936653719576e-10, 3.8636916787117e-9,     60055068.8360451]])